# Custom Loss


# import

In [ ]:
import sys
import time 
import math
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

sys.path.append("../../../")

%run ../../../ai/tf/setup/sess.py
%run ../../../ai/common/images.py
%run ../../../ai/nas/everyone/micro/model_inputs.py
%run ../../../ac/profile/measure_time.py

# load dataset

In [ ]:
tf_src_path = "/home/chy/archive-data/processed/everyone-tfr-candide"
tfrecord_files = glob.glob(os.path.join(tf_src_path, '*.tfr'))

batch_size = 8
for_lab = tfrecord_files[0:150]
data_iter = to_dataset(for_lab, batch_size=batch_size)
t_uid, t_frame, t_face, t_le, t_re, t_so, t_cam_x, t_cam_y, t_cam_to_x, t_cam_to_y, t_candide = data_iter.get_next()
tensors = [t_uid, t_frame, t_face, t_le, t_re, t_so, t_cam_x, t_cam_y, t_cam_to_x, t_cam_to_y, t_candide]

# Miscellaneous

In [ ]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.last_epoch_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.last_epoch_start)
        
    def on_train_end(self, logs={}):
        print('\n==> Total time elapsed: ', self.total_time())
        
    def total_time(self): 
        return sum(self.times)

# Load dataset

In [ ]:
size = 10000
x_train = []
y_train = []
x_test = []
y_test = []

with tf.Session(config=get_sess_config()) as sess:
    sess.run(data_iter.initializer)
    
    for i in range(size):
        _, _, _, _, _, so, cam_x, cam_y, cam_to_x, cam_to_y, candide = sess.run(tensors)
        for j in range(batch_size-1):
            X = np.split(candide[j], 20) + [cam_to_x[j], cam_to_y[j]]
            x_train.append(X)
            y_train.append((cam_x[j], cam_y[j]))

# model

In [ ]:
l2_reg = tf.keras.regularizers.l2
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
  
    tf.keras.layers.Dense(512, kernel_regularizer=l2_reg(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
timeHistory = TimeHistory()

# train


|#batch|loss|acc|val loss|val acc|
|-|-|-|-|-|
|1|
|2|
|4|


<br>

In [ ]:
tx = df_train_x.values
ty = df_train_y.values
vx = df_test_x.values
vy = df_test_y.values
model.fit(tx, ty, 
          validation_data=(vx, vy),
          callbacks=[timeHistory],
          batch_size=8,
          verbose=1, 
          epochs=5)